In [8]:
import os
import pandas as pd

In [9]:
processed_shutoffs = pd.read_csv(
    '../data/processed/processed-shutoffs.csv', index_col=0, dtype=str)
processed_shutoffs['deenergize_time_date'] = pd.to_datetime(processed_shutoffs['deenergize_time']).dt.date

weather = pd.DataFrame(columns = ['circuit_name', 'deenergize_time_date', 'tmin_d-4', 'tmax_d-4', 'wspd_d-4',
'tmin_d-3', 'tmax_d-3', 'wspd_d-3', 'tmin_d-2', 'tmax_d-2', 'wspd_d-2',
'tmin_d-1', 'tmax_d-1', 'wspd_d-1', 'tmin_d-0', 'tmax_d-0', 'wspd_d-0'])

raw_weather_files = os.listdir('../data/weather/raw/daily/')
for i in raw_weather_files:
    deenergize_time = pd.to_datetime(i.split('_')[2][0:-4])
    new_circuit_row = pd.DataFrame(columns = ['circuit_name', 'deenergize_time_date', 'tmin_d-4', 'tmax_d-4', 'wspd_d-4',
    'tmin_d-3', 'tmax_d-3', 'wspd_d-3', 'tmin_d-2', 'tmax_d-2', 'wspd_d-2',
    'tmin_d-1', 'tmax_d-1', 'wspd_d-1', 'tmin_d-0', 'tmax_d-0', 'wspd_d-0'], index=range(1))
    new_circuit_row['circuit_name'] = i.split('_')[1] 
    new_circuit_row['deenergize_time_date'] = deenergize_time
    temp_df = pd.read_csv('../data/weather/raw/daily/' + i)
    temp_df['time'] = pd.to_datetime(temp_df['time'])
    for j in range(len(temp_df)):
        # print(temp_df.iloc[j]['time'])
        if (deenergize_time - temp_df.iloc[j]['time']).days == 4:
            new_circuit_row['tmin_d-4'] = temp_df.iloc[j]['tmin']
            new_circuit_row['tmax_d-4'] = temp_df.iloc[j]['tmax']
            new_circuit_row['wspd_d-4'] = temp_df.iloc[j]['wspd']
            # new_circuit_row.extend([temp_df.iloc[j]['tmin'], temp_df.iloc[j]['tmax'], temp_df.iloc[j]['wspd']])
        elif (deenergize_time - temp_df.iloc[j]['time']).days == 3:
            new_circuit_row['tmin_d-3'] = temp_df.iloc[j]['tmin']
            new_circuit_row['tmax_d-3'] = temp_df.iloc[j]['tmax']
            new_circuit_row['wspd_d-3'] = temp_df.iloc[j]['wspd']
            # new_circuit_row.extend([temp_df.iloc[j]['tmin'], temp_df.iloc[j]['tmax'], temp_df.iloc[j]['wspd']])
        elif (deenergize_time - temp_df.iloc[j]['time']).days == 2:
            new_circuit_row['tmin_d-2'] = temp_df.iloc[j]['tmin']
            new_circuit_row['tmax_d-2'] = temp_df.iloc[j]['tmax']
            new_circuit_row['wspd_d-2'] = temp_df.iloc[j]['wspd']
            # new_circuit_row.extend([temp_df.iloc[j]['tmin'], temp_df.iloc[j]['tmax'], temp_df.iloc[j]['wspd']])
        elif (deenergize_time - temp_df.iloc[j]['time']).days == 1:
            new_circuit_row['tmin_d-1'] = temp_df.iloc[j]['tmin']
            new_circuit_row['tmax_d-1'] = temp_df.iloc[j]['tmax']
            new_circuit_row['wspd_d-1'] = temp_df.iloc[j]['wspd']
            # print(temp_df.iloc[j]['time'].date, 'deenergize-1', deenergize_time.date)
            # new_circuit_row.extend([temp_df.iloc[j]['tmin'], temp_df.iloc[j]['tmax'], temp_df.iloc[j]['wspd']])
        elif temp_df.iloc[j]['time'] == deenergize_time:
            new_circuit_row['tmin_d-0'] = temp_df.iloc[j]['tmin']
            new_circuit_row['tmax_d-0'] = temp_df.iloc[j]['tmax']
            new_circuit_row['wspd_d-0'] = temp_df.iloc[j]['wspd']
            # new_circuit_row.extend([temp_df.iloc[j]['tmin'], temp_df.iloc[j]['tmax'], temp_df.iloc[j]['wspd']])
    weather = pd.concat([weather, new_circuit_row], ignore_index=True)


In [10]:
processed_shutoffs.deenergize_time_date = pd.to_datetime(processed_shutoffs.deenergize_time_date, format='%Y-%m-%d %H:%M:%S')

In [11]:
# processed_shutoffs.info()

In [12]:
# weather.head(3)

processed_shutoffs_weather = processed_shutoffs.merge(weather, how = 'inner', left_on = ['circuit_name', 'deenergize_time_date'], 
right_on = ['circuit_name', 'deenergize_time_date'])

print("length of dataframe: ", len(processed_shutoffs_weather))
print(processed_shutoffs_weather.isna().sum())


length of dataframe:  1781
circuit_name              0
deenergize_time           0
restoration_time          0
time_out_min              0
key_communities           0
hftd_tier                 0
total_affected            0
residential_affected      0
zip_code                  0
longitude                 0
latitude                  0
substn_present            0
total_population          0
deenergize_time_date      0
tmin_d-4                 33
tmax_d-4                 33
wspd_d-4                312
tmin_d-3                 44
tmax_d-3                 44
wspd_d-3                359
tmin_d-2                 56
tmax_d-2                 56
wspd_d-2                310
tmin_d-1                113
tmax_d-1                113
wspd_d-1                363
tmin_d-0                208
tmax_d-0                208
wspd_d-0                460
dtype: int64


In [15]:
processed_shutoffs_weather.drop(columns='deenergize_time_date').to_csv('../data/processed/processed-shutoffs-weather.csv', index=False)